## Homework

In [ ]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX' -O data.csv

--2021-12-12 08:33:31--  https://docs.google.com/uc?export=download&id=1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX
Resolving docs.google.com (docs.google.com)... 64.233.191.113, 64.233.191.101, 64.233.191.102, ...
Connecting to docs.google.com (docs.google.com)|64.233.191.113|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-04-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/jh7flrphf7hmh65jesdeg4964vh1tt22/1639297950000/03856158561714992485/*/1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX?e=download [following]
--2021-12-12 08:33:32--  https://doc-04-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/jh7flrphf7hmh65jesdeg4964vh1tt22/1639297950000/03856158561714992485/*/1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX?e=download
Resolving doc-04-ak-docs.googleusercontent.com (doc-04-ak-docs.googleusercontent.com)... 209.85.147.132, 2607:f8b0:4001:c20::84
Connecting to doc-04-ak-docs.googleusercontent.com (doc-04-ak-d

В этой домашней работе вы будете предсказывать стоимость домов по их характеристикам.

Метрика качества: `RMSE`

Оценивание:
* Baseline - 2 балла
* Feature Engineering - 2 балла
* Model Selection - 3 балла
* Ensemble v.1 - 3 балла
* (*) Ensemble v.2 - дополнительно, 2 балла

### Описание датасета

Короткое описание данных:
```
price: sale price (this is the target variable)
id: transaction id
timestamp: date of transaction
full_sq: total area in square meters, including loggias, balconies and other non-residential areas
life_sq: living area in square meters, excluding loggias, balconies and other non-residential areas
floor: for apartments, floor of the building
max_floor: number of floors in the building
material: wall material
build_year: year built
num_room: number of living rooms
kitch_sq: kitchen area
state: apartment condition
product_type: owner-occupier purchase or investment
sub_area: name of the district

The dataset also includes a collection of features about each property's surrounding neighbourhood, and some features that are constant across each sub area (known as a Raion). Most of the feature names are self explanatory, with the following notes. See below for a complete list.

full_all: subarea population
male_f, female_f: subarea population by gender
young_*: population younger than working age
work_*: working-age population
ekder_*: retirement-age population
n_m_{all|male|female}: population between n and m years old
build_count_*: buildings in the subarea by construction type or year
x_count_500: the number of x within 500m of the property
x_part_500: the share of x within 500m of the property
_sqm_: square meters
cafe_count_d_price_p: number of cafes within d meters of the property that have an average bill under p RUB
trc_: shopping malls
prom_: industrial zones
green_: green zones
metro_: subway
_avto_: distances by car
mkad_: Moscow Circle Auto Road
ttk_: Third Transport Ring
sadovoe_: Garden Ring
bulvar_ring_: Boulevard Ring
kremlin_: City center
zd_vokzaly_: Train station
oil_chemistry_: Dirty industry
ts_: Power plant
```

### Setup

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv("data.csv", parse_dates=["timestamp"])


Разделите имеющиеся у вас данные на обучающую и тестовую выборки. В качестве обучающей выборки возьмите первые 80% данных, последние 20% - тестовая выборка.

# **Предобработка**



In [ ]:
df = pd.get_dummies(df)

In [ ]:
df

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_quota,preschool_education_centers_raion,children_school,school_quota,school_education_centers_raion,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25_raion,shopping_centers_raion,office_raion,full_all,male_f,female_f,young_all,young_male,young_female,work_all,work_male,work_female,ekder_all,...,sub_area_Veshnjaki,sub_area_Vnukovo,sub_area_Vojkovskoe,sub_area_Vostochnoe,sub_area_Vostochnoe Degunino,sub_area_Vostochnoe Izmajlovo,sub_area_Vyhino-Zhulebino,sub_area_Zamoskvorech'e,sub_area_Zapadnoe Degunino,sub_area_Zjablikovo,sub_area_Zjuzino,culture_objects_top_25_no,culture_objects_top_25_yes,thermal_power_plant_raion_no,thermal_power_plant_raion_yes,incineration_raion_no,incineration_raion_yes,oil_chemistry_raion_no,oil_chemistry_raion_yes,radiation_raion_no,radiation_raion_yes,railroad_terminal_raion_no,railroad_terminal_raion_yes,big_market_raion_no,big_market_raion_yes,nuclear_reactor_raion_no,nuclear_reactor_raion_yes,detention_facility_raion_no,detention_facility_raion_yes,water_1line_no,water_1line_yes,big_road1_1line_no,big_road1_1line_yes,railroad_1line_no,railroad_1line_yes,ecology_excellent,ecology_good,ecology_no data,ecology_poor,ecology_satisfactory
0,0,2014-12-26,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.293465e+06,77878,0.023464,0.195781,4713,2279.0,4,5212,10027.0,8,1,NaN,3,0,2,2,0,2,6,115352,52813,62539,10600,5422,5178,49882,24603,25279,17396,...,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,0,1,0
1,1,2012-10-04,64,64.0,16.0,NaN,NaN,NaN,NaN,NaN,NaN,6.677245e+07,9553,0.336177,0.072158,656,NaN,0,629,NaN,0,0,NaN,0,0,1,0,0,0,1,13890,6584,7307,1370,709,661,6127,3237,2890,2056,...,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,1,0,1,0,0,1,1,0,1,0,1,0,1,0,1,0,0,0,1,0,0
2,2,2014-02-05,83,44.0,9.0,17.0,1.0,1985.0,3.0,10.0,3.0,1.216448e+07,78507,0.297166,0.000000,4611,3092.0,7,5067,7478.0,7,0,NaN,2,0,14,1,0,6,4,67710,33382,34328,10175,5105,5070,48620,25579,23041,19712,...,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,1,1,0,1,0,1,0,0,1,0,0,0
3,3,2012-07-26,71,49.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,4.708040e+06,38075,0.271702,0.310199,2448,2080.0,3,2748,3885.0,4,0,NaN,0,0,0,0,0,0,1,85219,39138,46081,5572,2978,2594,22413,11273,11140,10090,...,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,0
4,4,2014-10-29,60,42.0,9.0,9.0,1.0,1970.0,3.0,6.0,2.0,1.428699e+07,157010,0.389354,0.194489,7751,5041.0,6,8004,11081.0,7,0,125.0,3,0,5,3,0,5,3,12327,5588,6739,16831,8637,8194,98260,47405,50855,41919,...,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,0,1,1,0,1,0,1,0,1,0,0,1,1,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19995,2014-09-22,45,27.0,2.0,9.0,1.0,1978.0,2.0,5.0,2.0,2.481385e+07,174831,0.683844,0.037178,10712,4172.0,7,11217,10559.0,7,0,2300.0,1,0,7,7,0,7,1,178264,82799,95465,23483,12083,11400,105171,52848,52323,46177,...,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,0,1,1,0,1,0,1,0,1,0,0,1,1,0,1,0,0,1,0,0,0
19996,19996,2013-12-06,38,NaN,4.0,17.0,1.0,NaN,2.0,1.0,1.0,6.677245e+07,9553,0.336177,0.072158,656,NaN,0,629,NaN,0,0,NaN,0,0,1,0,0,0,1,13890,6584,7307,1370,709,661,6127,3237,2890,2056,...,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,1,0,1,0,0,1,1,0,1,0,1,0,1,0,1,0,0,0,1,0,0
19997,19997,2014-06-26,35,14.0,9.0,22.0,1.0,2001.0,1.0,10.0,2.0,1.004686e+07,76156,0.071056,0.246032,3816,3150.0,5,3151,5519.0,5,0,2715.0,3,1,7,6,0,3,4,130229,60619,69610,7296,3649,3647,52450,26354,26096,16410,...,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,

In [ ]:
# заполнение нанов

nans = df.columns[df.isna().any()].tolist()
for item in nans:
  df[item].fillna(value = df[item].mean(), inplace = True)

Возможно в ваших моделях вам придется указывать, какие колонки являются категориальными (например, в бустингах). Для упрощения предлагается разделить колонки по следующему принципу:
```
drop_columns = [
    'id',           # May leak information
    'timestamp',    # May leak information
]
cat_columns = [
    'product_type',              #
    'material',                  # Material of the wall
    'state',                     # Satisfaction level
    'sub_area',                  # District name
    'culture_objects_top_25',    #
    'thermal_power_plant_raion', #
    'incineration_raion',        #
    'oil_chemistry_raion',       #
    'radiation_raion',           #
    'railroad_terminal_raion',   #
    'big_market_raion',          #
    'nuclear_reactor_raion',     #
    'detention_facility_raion',  #
    'ID_metro',                  #
    'ID_railroad_station_walk',  #
    'ID_railroad_station_avto',  #
    'water_1line',               #
    'ID_big_road1',              #
    'big_road1_1line',           #
    'ID_big_road2',              #
    'railroad_1line',            #
    'ID_railroad_terminal',      #
    'ID_bus_terminal',           #
    'ecology',                   #
]
num_columns = list(set(df.columns).difference(set(cat_columns + drop_columns)))
```

In [ ]:
drop_columns = [
    'id',           # May leak information
    'timestamp',    # May leak information
]
cat_columns = [
    'product_type',              #
    'material',                  # Material of the wall
    'state',                     # Satisfaction level
    'sub_area',                  # District name
    'culture_objects_top_25',    #
    'thermal_power_plant_raion', #
    'incineration_raion',        #
    'oil_chemistry_raion',       #
    'radiation_raion',           #
    'railroad_terminal_raion',   #
    'big_market_raion',          #
    'nuclear_reactor_raion',     #
    'detention_facility_raion',  #
    'ID_metro',                  #
    'ID_railroad_station_walk',  #
    'ID_railroad_station_avto',  #
    'water_1line',               #
    'ID_big_road1',              #
    'big_road1_1line',           #
    'ID_big_road2',              #
    'railroad_1line',            #
    'ID_railroad_terminal',      #
    'ID_bus_terminal',           #
    'ecology',                   #
]
num_columns = list(set(df.columns).difference(set(cat_columns + drop_columns)))
num_columns

['office_sqm_2000',
 'detention_facility_km',
 'build_count_1921-1945',
 'church_count_2000',
 'cafe_count_5000',
 'prom_part_2000',
 'cafe_count_5000_price_4000',
 'oil_chemistry_raion_no',
 'sub_area_Jakimanka',
 'sub_area_Vostochnoe',
 'cafe_avg_price_3000',
 'big_church_count_5000',
 'green_part_1500',
 'sub_area_Poselenie Moskovskij',
 'sub_area_Novo-Peredelkino',
 'sub_area_Krjukovo',
 'detention_facility_raion_yes',
 'cafe_count_1500_price_500',
 "sub_area_Krasnosel'skoe",
 'thermal_power_plant_raion_yes',
 'big_church_count_500',
 'sub_area_Jaroslavskoe',
 'catering_km',
 'sub_area_Severnoe',
 'sub_area_Nagatino-Sadovniki',
 'sport_count_2000',
 'nuclear_reactor_raion_no',
 'sub_area_Savelovskoe',
 'market_count_1000',
 'leisure_count_3000',
 'prom_part_5000',
 'ekder_male',
 'incineration_km',
 'green_part_500',
 'sub_area_Poselenie Desjonovskoe',
 'sub_area_Poselenie Novofedorovskoe',
 'railroad_1line_no',
 "sub_area_Kuz'minki",
 'work_all',
 'sub_area_Hovrino',
 'raion_build

In [ ]:
X = df.drop(['price'], axis = 1)
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [ ]:
X_train.drop(['id', 'timestamp'], axis = 1, inplace = True)
X_test.drop(['id', 'timestamp'], axis = 1, inplace = True)

### Baseline (2 балла)

В качестве Baseline обучите `DecisionTreeRegressor` из `sklearn`.

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import scale
from sklearn.metrics import mean_squared_error # squared = True отвечает за RMSE


regtree = DecisionTreeRegressor( )
regtree.fit(X_train, y_train)

DecisionTreeRegressor()

Проверьте качество на отложенной выборке.

In [ ]:
print('Common regtree score is {}'.format(mean_squared_error(regtree.predict(X_test), y_test, squared = False)))

Common regtree score is 3647881.127750655


### Feature Engineering (2 балла)

Часто улучшить модель можно с помощью аккуратного Feature Engineering.

Добавим в модель дополнительные признаки:
* "Как часто в этот год и этот месяц появлились объявления"
* "Как часто в этот год и эту неделю появлялись объявления"

In [ ]:
month_year = (df.timestamp.dt.month + df.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
df["month_year_cnt"] = month_year.map(month_year_cnt_map)

week_year = (df.timestamp.dt.weekofyear + df.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
df["week_year_cnt"] = week_year.map(week_year_cnt_map)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  """


Добавьте следюущие дополнительные признаки:
* Месяц (из колонки `timestamp`)
* День недели (из колонки `timestamp`)
* Отношение "этаж / максимальный этаж в здании" (колонки `floor` и `max_floor`)
* Отношение "площадь кухни / площадь квартиры" (колонки `kitchen_sq` и `full_sq`)

По желанию можно добавить и другие признаки.

In [ ]:
for item in df.index:
  df.loc[df.index[item], 'Month'] = df.loc[df.index[item], 'timestamp'].strftime("%m")
  df.loc[df.index[item], 'Day'] = df.loc[df.index[item], 'timestamp'].strftime("%d")
  df.loc[df.index[item], 'floor_relation'] = df.loc[df.index[item], 'floor'] / df.loc[df.index[item], 'max_floor']
  df.loc[df.index[item], 'sq_relation'] = df.loc[df.index[item], 'kitch_sq'] / df.loc[df.index[item], 'full_sq']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in double_scalars
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in double_scalars
  after removing the cwd from sys.path.


In [ ]:
df.columns[df.isna().any()].tolist()

['floor_relation', 'sq_relation']

In [ ]:
df['floor_relation'].fillna(value = df['floor_relation'].mean(), inplace = True)
df['sq_relation'].fillna(value = df['sq_relation'].mean(), inplace = True)

In [ ]:
df.columns[df.isna().any()].tolist()

[]

In [ ]:
df.drop(['id', 'timestamp'], axis = 1, inplace = True)


In [ ]:
df

,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_quota,preschool_education_centers_raion,children_school,school_quota,school_education_centers_raion,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25_raion,shopping_centers_raion,office_raion,full_all,male_f,female_f,young_all,young_male,young_female,work_all,work_male,work_female,ekder_all,ekder_male,ekder_female,...,sub_area_Vyhino-Zhulebino,sub_area_Zamoskvorech'e,sub_area_Zapadnoe Degunino,sub_area_Zjablikovo,sub_area_Zjuzino,culture_objects_top_25_no,culture_objects_top_25_yes,thermal_power_plant_raion_no,thermal_power_plant_raion_yes,incineration_raion_no,incineration_raion_yes,oil_chemistry_raion_no,oil_chemistry_raion_yes,radiation_raion_no,radiation_raion_yes,railroad_terminal_raion_no,railroad_terminal_raion_yes,big_market_raion_no,big_market_raion_yes,nuclear_reactor_raion_no,nuclear_reactor_raion_yes,detention_facility_raion_no,detention_facility_raion_yes,water_1line_no,water_1line_yes,big_road1_1line_no,big_road1_1line_yes,railroad_1line_no,railroad_1line_yes,ecology_excellent,ecology_good,ecology_no data,ecology_poor,ecology_satisfactory,month_year_cnt,week_year_cnt,Month,Day,floor_relation,sq_relation
0,1,1.000000,1.0,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,5.293465e+06,77878,0.023464,0.195781,4713,2279.000000,4,5212,10027.000000,8,1,1190.456871,3,0,2,2,0,2,6,115352,52813,62539,10600,5422,5178,49882,24603,25279,17396,5130,12266,...,0,0,0,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,0,1,0,1122,112,12,26,1.000000,1.000000
1,64,64.000000,16.0,12.603928,1.838505,3687.417575,1.91071,6.454917,2.106193,6.677245e+07,9553,0.336177,0.072158,656,3263.177788,0,629,8309.499014,0,0,1190.456871,0,0,1,0,0,0,1,13890,6584,7307,1370,709,661,6127,3237,2890,2056,583,1473,...,0,0,0,0,0,1,0,1,0,1,0,1,0,1,0,1,0,0,1,1,0,1,0,1,0,1,0,1,0,0,0,1,0,0,327,68,10,04,1.269446,0.100858
2,83,44.000000,9.0,17.000000,1.000000,1985.000000,3.00000,10.000000,3.000000,1.216448e+07,78507,0.297166,0.000000,4611,3092.000000,7,5067,7478.000000,7,0,1190.456871,2,0,14,1,0,6,4,67710,33382,34328,10175,5105,5070,48620,25579,23041,19712,6368,13344,...,0,0,0,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,1,1,0,1,0,1,0,0,1,0,0,0,752,192,02,05,0.529412,0.120482
3,71,49.000000,2.0,12.603928,1.838505,3687.417575,1.91071,6.454917,2.106193,4.708040e+06,38075,0.271702,0.310199,2448,2080.000000,3,2748,3885.000000,4,0,1190.456871,0,0,0,0,0,0,1,85219,39138,46081,5572,2978,2594,22413,11273,11140,10090,2911,7179,...,0,0,0,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,0,262,69,07,26,0.158681,0.090914
4,60,42.000000,9.0,9.000000,1.000000,1970.000000,3.00000,6.000000,2.000000,1.428699e+07,157010,0.389354,0.194489,7751,5041.000000,6,8004,11081.000000,7,0,125.000000,3,0,5,3,0,5,3,12327,5588,6739,16831,8637,8194,98260,47405,50855,41919,12424,29495,...,0,0,0,0,0,1,0,1,0,1,0,1,0,0,1,1,0,1,0,1,0,1,0,0,1,1,0,1,0,0,1,0,0,0,711,214,10,29,1.000000,0.100000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,45,27.000000,2.0,9.000000,1.000000,1978.000000,2.00000,5.000000,2.000000,2.481385e+07,174831,0.683844,0.037178,10712,4172.000000,7,11217,10559.000000,7,0,2300.000000,1,0,7,7,0,7,1,178264,82799,95465,23483,12083,11400,105171,52848,52323,46177,14748,31429,...,0,0,0,0,0,1,0,1,0,1,0,1,0,0,1,1,0,1,0,1,0,1,0,0,1,1,0,1,0,0,1,0,0,0,734,181,09,22,0.222222,0.111111
19996,38,34.032333,4.0,17.000000,1.000000,3687.417575,2.00000,1.000000,1.000000,6.677245e+07,9553,0.336177,0.072158,656,3263.177788,0,629,8309.499014,0,

In [ ]:
df.replace([np.inf, -np.inf], np.nan,inplace=True)

In [ ]:
df.columns[df.isna().any()].tolist()

['floor_relation']

In [ ]:
df['floor_relation'].fillna(value = df['floor_relation'].mean(), inplace = True)

Разделите выборку на обучающую и тестовую еще раз (потому что дополнительные признаки созданы для исходной выборки).

In [ ]:
X = df.drop(['price'], axis = 1)
y = df['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)



In [ ]:
tree2 = DecisionTreeRegressor( )
tree2.fit(X_train, y_train)


DecisionTreeRegressor()

In [ ]:
print('Regtree score is {}'.format(mean_squared_error(tree2.predict(X_test), y_test, squared = False)))

Regtree score is 3586589.921241382


### Model Selection (3 балла)

Посмотрите, какого качества можно добиться если использовать разные модели:
* `DecisionTreeRegressor` из `sklearn`
* `RandomForestRegressor` из `sklearn`
* `CatBoostRegressor`

Также вы можете попробовать линейные модели, другие бустинги (`LigthGBM` и `XGBoost`).

Почти все библиотеки поддерживают удобный способ подбора гиперпараметров: посмотрите как это делать в [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) или в [catboost](https://catboost.ai/docs/concepts/python-reference_catboostregressor_grid_search.html).

Проверяйте качество каждой модели на тестовой выборке и выберите наилучшую.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

forest = RandomForestRegressor(n_jobs = -1)
forest.fit(X_train, y_train)

RandomForestRegressor(n_jobs=-1)

In [ ]:
print('DecisionTreeRegressor score is {}'.format(mean_squared_error(tree2.predict(X_test), y_test, squared = False)))
print('RandomForestRegressor score is {}'.format(mean_squared_error(forest.predict(X_test), y_test, squared = False)))

DecisionTreeRegressor score is 3586589.921241382
RandomForestRegressor score is 2759277.9594834987


In [ ]:
!pip install catboost

     |████████████████████████████████| 76.3 MB 1.3 MB/s 


In [ ]:
from catboost import CatBoostRegressor

cat = CatBoostRegressor()
cat.fit(X_train, y_train)


Learning rate set to 0.06345
0:	learn: 4494557.0287663	total: 120ms	remaining: 2m
1:	learn: 4354119.5491062	total: 184ms	remaining: 1m 31s
2:	learn: 4226031.4908781	total: 256ms	remaining: 1m 24s
3:	learn: 4112761.1682807	total: 320ms	remaining: 1m 19s
4:	learn: 4001475.5975507	total: 380ms	remaining: 1m 15s
5:	learn: 3895314.9415894	total: 445ms	remaining: 1m 13s
6:	learn: 3798651.2339388	total: 507ms	remaining: 1m 11s
7:	learn: 3707227.0948752	total: 566ms	remaining: 1m 10s
8:	learn: 3627451.6229125	total: 626ms	remaining: 1m 8s
9:	learn: 3549722.7524032	total: 695ms	remaining: 1m 8s
10:	learn: 3477855.1814675	total: 762ms	remaining: 1m 8s
11:	learn: 3415149.1292869	total: 824ms	remaining: 1m 7s
12:	learn: 3356523.2888659	total: 888ms	remaining: 1m 7s
13:	learn: 3300345.1872549	total: 951ms	remaining: 1m 6s
14:	learn: 3248812.5111771	total: 1.01s	remaining: 1m 6s
15:	learn: 3204046.8959576	total: 1.08s	remaining: 1m 6s
16:	learn: 3157521.8459099	total: 1.13s	remaining: 1m 5s
17:	lear

In [ ]:
print('CatBoostRegressor score is {}'.format(mean_squared_error(cat.predict(X_test), y_test, squared = False)))
# Catboost показывает себя лучше всех 

CatBoostRegressor score is 2594394.393851057


### Ensemble v.1 (3 балла)

Ансамбли иногда оказываются лучше чем одна большая модель.

В колонке `product_type` содержится информация о том, каким является объявление: `Investment` (продажа квартиры как инвестиции) или `OwnerOccupier` (продажа квартиры для жилья). Логично предположить, что если сделать по модели на каждый из этих типов, то качество будет выше.

Обучите свои лучшие модели на отдельно на `Investment` и `OwnerOccupier` (т.е. у вас будет `model_invest`, обученная на `(invest_train_X, invest_train_Y)` и `model_owner`, обученная на `(owner_train_X, owner_train_Y)`) и проверьте качество на отложенной выборке (т.е. на исходном `test_split`).

In [ ]:
for item in df.columns.tolist():
  if 'Invest' in item:
    print(item)
    break

product_type_Investment


In [ ]:
X_invest = df[df['product_type_Investment'] == 1].drop(['price'], axis = 1)
y_invest = df[df['product_type_Investment'] == 1]['price']

X_train_inv, X_test_inv, y_train_inv, y_test_inv = train_test_split(X_invest, y_invest, test_size = 0.2)

X_owner = df[df['product_type_Investment'] == 0].drop(['price'], axis = 1)
y_owner = df[df['product_type_Investment'] == 0]['price']

X_train_own, X_test_own, y_train_own, y_test_own = train_test_split(X_owner, y_owner, test_size = 0.2)

In [ ]:
from sklearn.ensemble import BaggingRegressor

cat_invest = CatBoostRegressor()

model_invest = BaggingRegressor(base_estimator = cat_invest)
model_invest.fit(X_train_inv, y_train_inv)

invest_pred = model_invest.predict(X_test_inv)


Выходные данные были обрезаны до нескольких последних строк (5000).
4:	learn: 4270623.1246898	total: 279ms	remaining: 55.6s
5:	learn: 4180519.8925889	total: 332ms	remaining: 55s
6:	learn: 4084720.6014037	total: 397ms	remaining: 56.3s
7:	learn: 4004524.5652578	total: 461ms	remaining: 57.1s
8:	learn: 3931098.5911007	total: 513ms	remaining: 56.5s
9:	learn: 3851527.3889209	total: 562ms	remaining: 55.6s
10:	learn: 3784313.0146960	total: 611ms	remaining: 54.9s
11:	learn: 3731176.9928681	total: 661ms	remaining: 54.4s
12:	learn: 3670784.9804664	total: 726ms	remaining: 55.1s
13:	learn: 3616740.0735355	total: 777ms	remaining: 54.7s
14:	learn: 3566151.6863896	total: 827ms	remaining: 54.3s
15:	learn: 3517074.0385609	total: 877ms	remaining: 54s
16:	learn: 3474334.5297535	total: 937ms	remaining: 54.2s
17:	learn: 3434999.6842101	total: 993ms	remaining: 54.2s
18:	learn: 3394964.4133774	total: 1.05s	remaining: 54.1s
19:	learn: 3361493.3993422	total: 1.1s	remaining: 53.9s
20:	learn: 3328678.7110658	tota

In [ ]:
cat_own = CatBoostRegressor()

model_own = BaggingRegressor(base_estimator = cat_own)
model_own.fit(X_train_own, y_train_own)

own_pred = model_own.predict(X_test_own)

Выходные данные были обрезаны до нескольких последних строк (5000).
4:	learn: 3468650.9375675	total: 199ms	remaining: 39.6s
5:	learn: 3349853.3621018	total: 247ms	remaining: 40.9s
6:	learn: 3235110.9401020	total: 292ms	remaining: 41.4s
7:	learn: 3131751.8092131	total: 328ms	remaining: 40.7s
8:	learn: 3039193.4858531	total: 365ms	remaining: 40.2s
9:	learn: 2952582.8966939	total: 401ms	remaining: 39.7s
10:	learn: 2864407.5664784	total: 448ms	remaining: 40.3s
11:	learn: 2782810.6274934	total: 483ms	remaining: 39.8s
12:	learn: 2708527.6717705	total: 519ms	remaining: 39.4s
13:	learn: 2634884.9434221	total: 556ms	remaining: 39.2s
14:	learn: 2560373.2560223	total: 592ms	remaining: 38.9s
15:	learn: 2485762.0562413	total: 646ms	remaining: 39.7s
16:	learn: 2424456.7866209	total: 682ms	remaining: 39.5s
17:	learn: 2364745.6326795	total: 718ms	remaining: 39.2s
18:	learn: 2308007.1825795	total: 754ms	remaining: 38.9s
19:	learn: 2253842.3767055	total: 791ms	remaining: 38.7s
20:	learn: 2199695.3499047

In [ ]:
invest_pred = pd.Series(invest_pred)
own_pred = pd.Series(own_pred)


In [ ]:
y_test_inv 

15223     1000000
8444      1000000
6775      1000000
2251     18400000
6294      7100000
           ...   
16910    10200000
9416      7250000
19898     5414200
3102     10900000
6554      5400000
Name: price, Length: 2568, dtype: int64

In [ ]:
unite_test = pd.concat([y_test_inv, y_test_own])
unite_pred = np.concatenate([invest_pred, own_pred])

In [ ]:
print("Ensemble result is {}".format(mean_squared_error(unite_test, unite_pred, squared = False)))

Ensemble result is 2567057.0499002235


In [ ]:
from sklearn.linear_model import HuberRegressor

### (*) Ensemble v.2 (дополнительно, 2 балла)

Попробуйте сделать для `Investment` более сложную модель: обучите `CatBoostRegressor` и `HuberRegressor` из `sklearn`, а затем сложите их предсказания с весами `w_1` и `w_2` (выберите веса сами; сумма весов равняется 1).

In [ ]:
import plotly.express as px

In [ ]:
weights = np.arange(0, 1, 0.1)
score = []
for weight in weights:
  cat = CatBoostRegressor( )
  huber = HuberRegressor( )
  cat.fit(X_train_inv, y_train_inv)
  huber.fit(X_train_inv, y_train_inv)
  pred = weight * cat.predict(X_test_inv) + (1 - weight) * huber.predict(X_test_inv)
  score.append(mean_squared_error(y_test_inv, pred, squared = False))

Learning rate set to 0.059159
0:	learn: 4832295.8997730	total: 70.9ms	remaining: 1m 10s
1:	learn: 4705194.5031820	total: 125ms	remaining: 1m 2s
2:	learn: 4579079.2416321	total: 177ms	remaining: 58.9s
3:	learn: 4465373.4526940	total: 235ms	remaining: 58.4s
4:	learn: 4369233.1656989	total: 287ms	remaining: 57.2s
5:	learn: 4273939.4581656	total: 349ms	remaining: 57.7s
6:	learn: 4187313.7860864	total: 406ms	remaining: 57.6s
7:	learn: 4100795.2810112	total: 465ms	remaining: 57.6s
8:	learn: 4021751.6032067	total: 519ms	remaining: 57.1s
9:	learn: 3947647.0234379	total: 574ms	remaining: 56.8s
10:	learn: 3876377.8681503	total: 636ms	remaining: 57.1s
11:	learn: 3813346.3432793	total: 693ms	remaining: 57s
12:	learn: 3753785.9660524	total: 761ms	remaining: 57.8s
13:	learn: 3702216.2910160	total: 818ms	remaining: 57.6s
14:	learn: 3651988.9204404	total: 873ms	remaining: 57.3s
15:	learn: 3605519.0245723	total: 931ms	remaining: 57.2s
16:	learn: 3559702.7518079	total: 984ms	remaining: 56.9s
17:	learn: 

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Learning rate set to 0.059159
0:	learn: 4832295.8997730	total: 80.7ms	remaining: 1m 20s
1:	learn: 4705194.5031820	total: 140ms	remaining: 1m 10s
2:	learn: 4579079.2416321	total: 198ms	remaining: 1m 5s
3:	learn: 4465373.4526940	total: 253ms	remaining: 1m 2s
4:	learn: 4369233.1656989	total: 305ms	remaining: 1m
5:	learn: 4273939.4581656	total: 361ms	remaining: 59.8s
6:	learn: 4187313.7860864	total: 425ms	remaining: 1m
7:	learn: 4100795.2810112	total: 481ms	remaining: 59.6s
8:	learn: 4021751.6032067	total: 534ms	remaining: 58.8s
9:	learn: 3947647.0234379	total: 592ms	remaining: 58.6s
10:	learn: 3876377.8681503	total: 650ms	remaining: 58.5s
11:	learn: 3813346.3432793	total: 709ms	remaining: 58.4s
12:	learn: 3753785.9660524	total: 766ms	remaining: 58.1s
13:	learn: 3702216.2910160	total: 821ms	remaining: 57.8s
14:	learn: 3651988.9204404	total: 880ms	remaining: 57.8s
15:	learn: 3605519.0245723	total: 945ms	remaining: 58.1s
16:	learn: 3559702.7518079	total: 1.01s	remaining: 58.2s
17:	learn: 351

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Learning rate set to 0.059159
0:	learn: 4832295.8997730	total: 71.8ms	remaining: 1m 11s
1:	learn: 4705194.5031820	total: 129ms	remaining: 1m 4s
2:	learn: 4579079.2416321	total: 181ms	remaining: 1m
3:	learn: 4465373.4526940	total: 245ms	remaining: 1m 1s
4:	learn: 4369233.1656989	total: 298ms	remaining: 59.3s
5:	learn: 4273939.4581656	total: 352ms	remaining: 58.4s
6:	learn: 4187313.7860864	total: 408ms	remaining: 57.9s
7:	learn: 4100795.2810112	total: 470ms	remaining: 58.3s
8:	learn: 4021751.6032067	total: 528ms	remaining: 58.1s
9:	learn: 3947647.0234379	total: 583ms	remaining: 57.8s
10:	learn: 3876377.8681503	total: 641ms	remaining: 57.6s
11:	learn: 3813346.3432793	total: 699ms	remaining: 57.6s
12:	learn: 3753785.9660524	total: 761ms	remaining: 57.8s
13:	learn: 3702216.2910160	total: 814ms	remaining: 57.3s
14:	learn: 3651988.9204404	total: 867ms	remaining: 57s
15:	learn: 3605519.0245723	total: 937ms	remaining: 57.6s
16:	learn: 3559702.7518079	total: 994ms	remaining: 57.5s
17:	learn: 351

...

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Learning rate set to 0.059159
0:	learn: 4832295.8997730	total: 72.6ms	remaining: 1m 12s
1:	learn: 4705194.5031820	total: 154ms	remaining: 1m 16s
2:	learn: 4579079.2416321	total: 215ms	remaining: 1m 11s
3:	learn: 4465373.4526940	total: 282ms	remaining: 1m 10s
4:	learn: 4369233.1656989	total: 339ms	remaining: 1m 7s
5:	learn: 4273939.4581656	total: 395ms	remaining: 1m 5s
6:	learn: 4187313.7860864	total: 468ms	remaining: 1m 6s
7:	learn: 4100795.2810112	total: 524ms	remaining: 1m 4s
8:	learn: 4021751.6032067	total: 582ms	remaining: 1m 4s
9:	learn: 3947647.0234379	total: 638ms	remaining: 1m 3s
10:	learn: 3876377.8681503	total: 700ms	remaining: 1m 2s
11:	learn: 3813346.3432793	total: 756ms	remaining: 1m 2s
12:	learn: 3753785.9660524	total: 815ms	remaining: 1m 1s
13:	learn: 3702216.2910160	total: 872ms	remaining: 1m 1s
14:	learn: 3651988.9204404	total: 937ms	remaining: 1m 1s
15:	learn: 3605519.0245723	total: 995ms	remaining: 1m 1s
16:	learn: 3559702.7518079	total: 1.05s	remaining: 1m
17:	learn

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Learning rate set to 0.059159
0:	learn: 4832295.8997730	total: 74.1ms	remaining: 1m 14s
1:	learn: 4705194.5031820	total: 129ms	remaining: 1m 4s
2:	learn: 4579079.2416321	total: 181ms	remaining: 1m
3:	learn: 4465373.4526940	total: 248ms	remaining: 1m 1s
4:	learn: 4369233.1656989	total: 302ms	remaining: 1m
5:	learn: 4273939.4581656	total: 359ms	remaining: 59.5s
6:	learn: 4187313.7860864	total: 416ms	remaining: 59s
7:	learn: 4100795.2810112	total: 483ms	remaining: 59.9s
8:	learn: 4021751.6032067	total: 538ms	remaining: 59.2s
9:	learn: 3947647.0234379	total: 596ms	remaining: 59s
10:	learn: 3876377.8681503	total: 651ms	remaining: 58.5s
11:	learn: 3813346.3432793	total: 722ms	remaining: 59.4s
12:	learn: 3753785.9660524	total: 778ms	remaining: 59.1s
13:	learn: 3702216.2910160	total: 833ms	remaining: 58.7s
14:	learn: 3651988.9204404	total: 887ms	remaining: 58.3s
15:	learn: 3605519.0245723	total: 955ms	remaining: 58.7s
16:	learn: 3559702.7518079	total: 1.02s	remaining: 58.9s
17:	learn: 3515661.

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [ ]:
fig = px.scatter(x = weights, y = score)
fig.update_layout(title = 'rmse for (0, 1) weights')